# API 的获取


在 2025 年的今天，获取所有渠道的 API 都已不再是什么大问题。


In [ ]:
# 在代码中安全引入 API key 的方式

新建 .env 文件用于项目环境下的 API 配置


# 如何在 Deno 中导入 npm 包？


Deno 现支持三种导入第三方包的方式：jsr、npm 或 https，详见[模块和依赖项](https://docs.deno.com/runtime/fundamentals/modules/)。

- jsr(JavaScript Registry)：这是 Deno 官方推荐和维护的包注册表、专门为 Deno 优化，提供更好的类型支持和安全性
- npm: (Node Package Manager)：允许直接使用 npm 包、Deno 会自动处理依赖关系和类型定义
- https://直接从 URL 导入模块、最原始的 Deno 导入方式、可以直接引用任何托管在网络上的 JavaScript/TypeScript 文件
  示例代码如下：

```js
import { camelCase } from "jsr:@luca/cases@1.0.0";
import { say } from "npm:cowsay@1.6.0";
import { pascalCase } from "https://deno.land/x/case/mod.ts";
```


由于 LangChain 和 OpenAI SDK 目前还没有在 JSR 上发布，我们可以继续使用 npm: 导入，或者使用 https://esm.sh/ 这样的 ESM CDN。比如`import { OpenAI } from "https://esm.sh/openai@4.28.4";`


In [1]:
import { HumanMessage } from "npm:@langchain/core/messages"
import { ChatOpenAI } from "npm:@langchain/openai"
import { OpenAI } from "npm:openai"


# 获取模型列表


这里要补充一些前置知识，即 OpenAI 推出的 SDK 已经成为了普适的 LLM API 标准，其 js 版为[openai/openai-node](https://github.com/openai/openai-node)。openai-node 提供了对 LLM 的底层访问能力；而 langchian 则提供了高级抽象和工具链。而获取模型列表就需要这样一种底层访问能力。


In [2]:
// 初始化 OpenAI SDK 客户端
const openai = new OpenAI({
  baseURL: "https://yunwu.ai/v1",
  apiKey: Deno.env.get("OPENAI_API_KEY"),
})


In [3]:
// 获取可用模型列表
console.log("获取模型列表中...")
const models = await openai.models.list()
console.log("可用的模型：")
models.data.forEach((model) => {
  console.log(`- ${model.id}`)
})


获取模型列表中...
可用的模型：
- o3-mini
- text-embedding-3-small
- gpt-4o-mini


In [4]:
models


ModelsPage {
  options: { method: "get", path: "/models" },
  response: Response {
    body: ReadableStream { locked: true },
    bodyUsed: true,
    headers: Headers {
      "content-type": "application/json; charset=utf-8",
      date: "Tue, 04 Mar 2025 13:20:31 GMT",
      server: "nginx",
      "strict-transport-security": "max-age=31536000",
      vary: "Accept-Encoding",
      "x-oneapi-request-id": "202503042120311273337385vI5G5Rv"
    },
    ok: true,
    redirected: false,
    status: 200,
    statusText: "OK",
    url: "https://yunwu.ai/v1/models"
  },
  body: {
    data: [
      {
        id: "o3-mini",
        object: "model",
        created: 1626777600,
        owned_by: "openai",
        permission: [ [Object] ],
        root: "o3-mini",
        parent: null
      },
      {
        id: "text-embedding-3-small",
        object: "model",
        created: 1626777600,
        owned_by: "openai",
        permission: [ [Object] ],
        root: "text-embedding-3-small",
     

# 使用 langchain


作为一个框架，LangChain 提供了一个抽象层，可以让我们更容易地使用各种 LLM（大语言模型）服务，包括但不限于 OpenAI 及其他 API 服务。LangChain 封装了底层 API 调用，提供了更高级的功能，如：链式调用（Chains）、记忆管理（Memory）、代理（Agents）、文档处理、提示词管理等


在项目代码文件中，Deno 可能会提示包不存在，这是我们需要执行`deno add npm:openai`来下载对应的包（其实是修改了 deno.json 文件，Deno 也会在运行时自动解析和抓取所需的模块）。


In [5]:
// 初始化 LangChain 的 ChatOpenAI
const chatModel = new ChatOpenAI({
  configuration: {
    baseURL: "https://yunwu.ai/v1",
  },
  modelName: "gpt-4o-mini", // 使用可用的模型
})


你可能会注意到这里并未显式给出 API-key，这是因为 Langchain 支持读取系统环境？？


In [7]:
// 使用 LangChain 进行对话测试
console.log("\n开始对话测试...")
const response = await chatModel.invoke([
  new HumanMessage("你好！请介绍一下你自己。"),
])

console.log("AI 回复:", response.content)



开始对话测试...
AI 回复: 你好！我是一个人工智能助手，旨在提供信息和帮助解决问题。我可以回答各种问题、提供建议、帮助学习新知识，或者与您进行有趣的对话。如果您有任何特定的问题或主题，欢迎随时告诉我！


成功！
